In [106]:
import numpy as np
import pandas as pd
import statistics
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# from pattern.en import spelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from pandas import DataFrame
!pip3 install imblearn
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import SGDClassifier

In [107]:
train_col_names = ['qid', 'question_text', 'target']
train_data = pd.read_csv('train.csv', names=train_col_names, skiprows=1)
train_data.head()

,qid,question_text,target
0,6f47b0f60633c2056455,"How can I reply to this comment, ""India is poo...",0
1,d49b3966070b27bf07fc,What did they use for transportation in Ancien...,0
2,6d5faa49380557c8ca7b,What are the most important provisions of Obam...,0
3,cebea75faa47388edcf5,At what age do most Finns master English today?,0
4,2a7b76a679cadb0a016e,What is cheapest place to live in India for on...,0


In [108]:
train_data.tail()

,qid,question_text,target
783668,8462d2ead608fe1dcf9b,What causes a lump on the back of your head?,0
783669,12d225a4e28317c02151,What is standalone in simple way?,0
783670,03edc045cadbb11558dd,How do you prevent having low potassium while ...,0
783671,c6fb7dc8d0b34160dbe4,What is it like to meet the band Imagine Dragons?,0
783672,6a58b525dc9ccaf25b72,"How can I find an intelligent, well reasoned p...",1


In [109]:
train_ids = train_data.qid.tolist()
train_questions = train_data.question_text.tolist()

In [110]:
train_labels = train_data.target.values.tolist()

In [111]:
print(train_ids[:5])
print(train_questions[:5])
print(train_labels[:5])

['6f47b0f60633c2056455', 'd49b3966070b27bf07fc', '6d5faa49380557c8ca7b', 'cebea75faa47388edcf5', '2a7b76a679cadb0a016e']
['How can I reply to this comment, "India is poor. It is a fact. I don\'t understand the unnecessary criticism of Snapchat CEO\'s statement"?', 'What did they use for transportation in Ancient India?', 'What are the most important provisions of Obamacare?', 'At what age do most Finns master English today?', 'What is cheapest place to live in India for one month which best suits for tourism also?']
[0, 0, 0, 0, 0]


In [112]:
test_col_names = ['qid', 'question_text']
test_data = pd.read_csv('test.csv', names=test_col_names, skiprows=1)
test_data.head()

,qid,question_text
0,f56a9a31974dc66186e8,Is it a good idea to go through a phlebotomy c...
1,d957c3758060f45da303,How can I fix a lead into a camlin compass to ...
2,ad822d5abaedb9e247b9,How many animes are there in world?
3,4e979c23eeb6a4bd1f2e,How do I tell my family I cut?
4,333cc031262566b8da49,How do I save down my bitcoin image address fr...


In [113]:
test_data.tail()

,qid,question_text
522444,e8e6aa5226f36c27fe41,"In terms of absolute passion and leadership, w..."
522445,015fd068afcb9d0b4007,How do I make my orthodox Indian parents accep...
522446,9f0ef49eff6a3ff9e735,How can air force officers go to ranger school?
522447,d6b02f52f76dc4c22afd,Why does my brothers cow kick me all the time?
522448,132ef601b08de269aee9,How was Japan's national animal chosen?


In [114]:
test_ids = test_data.qid.tolist()
test_questions = test_data.question_text.tolist()

In [115]:
print(test_ids[:5])
print(test_questions[:5])

['f56a9a31974dc66186e8', 'd957c3758060f45da303', 'ad822d5abaedb9e247b9', '4e979c23eeb6a4bd1f2e', '333cc031262566b8da49']
['Is it a good idea to go through a phlebotomy class before going into nursing school next year? Will it give me any advantage at all?', 'How can I fix a lead into a camlin compass to draw arcs, circles in engineering graphics?', 'How many animes are there in world?', 'How do I tell my family I cut?', 'How do I save down my bitcoin image address from coinbase?']


In [116]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

train_questions_new = []
test_questions_new = []

for sample_question in train_questions:
  sample_question_temp = sample_question.lower()
  train_questions_new.append(tokenizer.tokenize(sample_question_temp))

for sample_question in test_questions:
  sample_question_temp = sample_question.lower()
  test_questions_new.append(tokenizer.tokenize(sample_question_temp))

In [117]:
print(train_questions_new[:5])
print(test_questions_new[:5])

[['how', 'can', 'i', 'reply', 'to', 'this', 'comment', 'india', 'is', 'poor', 'it', 'is', 'a', 'fact', 'i', 'don', 't', 'understand', 'the', 'unnecessary', 'criticism', 'of', 'snapchat', 'ceo', 's', 'statement'], ['what', 'did', 'they', 'use', 'for', 'transportation', 'in', 'ancient', 'india'], ['what', 'are', 'the', 'most', 'important', 'provisions', 'of', 'obamacare'], ['at', 'what', 'age', 'do', 'most', 'finns', 'master', 'english', 'today'], ['what', 'is', 'cheapest', 'place', 'to', 'live', 'in', 'india', 'for', 'one', 'month', 'which', 'best', 'suits', 'for', 'tourism', 'also']]
[['is', 'it', 'a', 'good', 'idea', 'to', 'go', 'through', 'a', 'phlebotomy', 'class', 'before', 'going', 'into', 'nursing', 'school', 'next', 'year', 'will', 'it', 'give', 'me', 'any', 'advantage', 'at', 'all'], ['how', 'can', 'i', 'fix', 'a', 'lead', 'into', 'a', 'camlin', 'compass', 'to', 'draw', 'arcs', 'circles', 'in', 'engineering', 'graphics'], ['how', 'many', 'animes', 'are', 'there', 'in', 'world']

In [118]:
word_dict = {}
word_index = 0

In [119]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
print(stop_words)
print(len(stop_words))

{"haven't", 'an', 'needn', 'not', 'most', 'or', 'isn', 'through', 'them', 'of', 'as', 'by', 'should', 'm', 'into', "it's", 'ain', 'shouldn', 'my', 'has', 'which', 'these', 'this', 'there', 'doesn', 'himself', 's', 'all', "aren't", "don't", 'its', 'can', 'wouldn', 'same', "hasn't", "couldn't", 'hers', 'themselves', 'do', 'any', 'was', 'myself', 'we', 'so', "mightn't", 'his', 'before', 'yours', 'haven', 'few', 'her', 'they', 'wasn', 'about', 'but', 'were', 'after', 'then', "needn't", 'won', 'weren', 'while', "you'd", 'some', "won't", 'above', 'up', 'those', 'herself', 'am', 'if', 'o', 'hasn', 'our', 'for', 'once', "shan't", 'had', "hadn't", 't', 'on', 'because', 'are', 'who', 'couldn', 'you', 'their', 'theirs', 'what', 'i', 'mustn', 'further', 'out', 'the', 'and', 'with', 'each', 'why', 'now', 'that', 'from', 'over', 'did', 'below', "she's", "that'll", 'he', "doesn't", 'itself', 'been', "you're", 'again', 'other', 'down', 'a', 'hadn', 'y', "you'll", 'at', 'during', "wouldn't", 'until', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sowmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [120]:
def clean_stemm_stopwords(input_list):
  filtered_op = []

  for words_in_temp_sample in input_list:
    temp_sample = ''
    for word in words_in_temp_sample:
      if not word in stop_words and not word in temp_sample:
        temp_word = ps.stem(word)
        temp_sample += (temp_word + ' ')
        if temp_word not in word_dict.keys():
          word_dict[temp_word] = len(word_dict)
          
    temp_sample = temp_sample[:-1]
    filtered_op.append(temp_sample)

  return filtered_op

In [121]:
train_questions_filtered = clean_stemm_stopwords(train_questions_new)
test_questions_filtered = clean_stemm_stopwords(test_questions_new)

In [122]:
freq = np.asarray([np.zeros(len(word_dict)), np.zeros(len(word_dict))])

In [123]:
for x,y in zip(train_questions_filtered, train_labels):
    for w in x.split():
        freq[y, word_dict[w]] += 1

In [124]:
def feature_extraction(l):
    x = np.asarray([np.ones(len(l)), np.zeros(len(l)), np.zeros(len(l))])
    for i, s in enumerate(l):
        for w in s.split():
            x[1, i] += freq[0, word_dict[w]]
            x[2, i] += freq[1, word_dict[w]]
    return x

In [125]:
xtrain = feature_extraction(train_questions_filtered)
xsubmission = feature_extraction(test_questions_filtered)

In [126]:
xtrain = xtrain.transpose()

In [127]:
train_labels = np.asarray(train_labels)

In [128]:
train_labels.shape

(783673,)

In [129]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
!pip3 install xgboost
from xgboost import XGBClassifier

In [130]:
xtrain, xtest, ytrain, ytest = train_test_split(xtrain, train_labels, test_size = 0.1)

In [131]:
stc = StandardScaler()
xtrain = stc.fit_transform(xtrain)
xtest = stc.transform(xtest)


In [132]:
xsubmission = stc.transform(xsubmission.transpose())

In [133]:
smote = SMOTE(random_state=777,k_neighbors=5)
X_smote,y_smote = smote.fit_sample(xtrain,ytrain)
X_train, x_test, Y_train, y_test = train_test_split(X_smote,y_smote,test_size=0.3,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)

SGDClassifier()

In [134]:
from sklearn.metrics import classification_report

In [135]:
pred_sgd = sgd.predict(x_test)
print(accuracy_score(pred_sgd,y_test))
print(classification_report(y_test,pred_sgd))

0.7728585929569767
              precision    recall  f1-score   support

           0       0.72      0.88      0.80    198504
           1       0.85      0.66      0.74    198513

    accuracy                           0.77    397017
   macro avg       0.79      0.77      0.77    397017
weighted avg       0.79      0.77      0.77    397017



In [136]:
f1_score(y_test, pred_sgd)

0.7442898420863095

In [137]:
sub_df = pd.read_csv('sample_submission.csv')

In [138]:
sub_df['target'] = sgd.predict(xsubmission)

In [139]:
sub_df.to_csv('sub3.csv', index = False)


In [140]:
sgd.predict(xsubmission)

array([0, 0, 1, ..., 0, 0, 0])

In [141]:
a=np.array(sgd.predict(xsubmission))

In [142]:
a.sum()

78075

In [143]:
a.shape

(522449,)